In [1]:
import pandas as pd
import numpy as np
from eng_module import utils

# Concrete

In [3]:
def extract_RAM_conc_column_data(filepath: str) -> dict[str, list[str]]:
    """
    Returns a dictionary of relevant column design data from the RAM Concrete
    Column "Column Design" csv.
    """
    raw_data = utils.read_csv_file(filepath)

    level = []
    grid_loc = []
    size = []
    rebar = []
    fpc = []
    lux = []
    luy = []
    kx = []
    ky = []
    pu = []
    mu_x_top = []
    mu_y_top = []
    mu_x_bot = []
    mu_y_bot = []

    for idx, row in enumerate(raw_data):
        # multiIndex keys
        if "Level." in row:
            level.append(row[1])
        if "Grid Location:." in row:
            grid_loc.append(row[-1])

        # corresponding values for above keys
        if "Size:." in row:
            item = row[1].split('  ')[0].strip()
            size.append(item)
        if "Longitudinal:." in row:
            rebar.append(row[1].strip().split(' ')[0])
        if "f'c (ksi):." in row:
            fpc.append(row[1].strip())
        if "Unbraced Length (ft)." in row:
            lux.append(row[1])
            luy.append(row[2])
        if "K." in row:
            kx.append(row[1].strip())
            ky.append(row[2].strip())
        if "Axial" in row:
            pu.append(row[-1])
        if "Moment" in row and "Top" in row:
            mu_x_top.append(row[-1].strip())
            mu_y_top.append(raw_data[idx+1][-1].strip())
        if "Moment" in row and "Bottom" in row:
            mu_x_bot.append(row[-1].strip())
            mu_y_bot.append(raw_data[idx+1][-1].strip())

    column_data = {"level": level,
                   "grid_loc": grid_loc,
                   "size": size,
                   "rebar": rebar,
                   "fpc": fpc,
                   "lux": lux,
                   "luy": luy,
                   "kx": kx,
                   "ky": ky,
                   "pu": pu,
                   "mu_x_top": mu_x_top,
                   "mu_y_top": mu_y_top,
                   "mu_x_bot": mu_x_bot,
                   "mu_y_bot": mu_y_bot
                  }

    print(len(level), len(grid_loc), len(size), len(rebar), len(fpc), len(lux),
          len(luy), len(kx), len(ky), len(pu), len(mu_x_top), len(mu_y_top),
          len(mu_x_bot), len(mu_y_bot))

    return column_data

In [4]:
def create_RAM_concrete_column_schedule(column_data: dict[str, list[str]], 
                    xlsx: bool = False, 
                    output_filename: str = "RAM_Concrete_Column_Schedule.xlsx") -> pd.DataFrame:
    """
    Returns a column schedule from the dictionary created from the 
    extract_RAM_conc_column_data() function.

    If xlsx = True, it will also produce an Excel file.
    """
    levels_and_loc_dict = {k: col_data[k] for k in ('level', 'grid_loc') if k in col_data}
    col_sched_dict = {k: col_data[k] for k in ('size', 'rebar', 'fpc', 'pu') if k in col_data}

    mi_idx = pd.MultiIndex.from_frame(pd.DataFrame(levels_and_loc_dict))
    col_sched_df = pd.DataFrame(col_sched_dict)
    col_sched_df.index = mi_idx

    col_sched_df = col_sched_df.transpose()

    final = []
    for story in pd.Series(col_data['level']).unique():
        story_df = col_sched_df.loc[:, story]
        story_df['story'] = story
        final.append(story_df)
    schedule_df = pd.concat(final)


    schedule_df = schedule_df.reindex(schedule_df.columns.sort_values(), axis=1)
    story = schedule_df.story
    schedule_df = schedule_df.drop(columns=['story'])
    schedule_df.insert(loc=0, column='story', value=story)

    if xlsx:
        schedule_df.to_excel(output_filename)

    return schedule_df

In [5]:
col_data = extract_RAM_conc_column_data(r'/Users/danki/pycode/eng_module/test_data/RAM_Lateral_Concrete_Designs.csv')

1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211 1211


In [6]:
levels_and_loc_dict = {
        k: col_data[k] for k in ("level", "grid_loc") if k in col_data
    }
col_sched_dict = {
    k: col_data[k] for k in ("size", "rebar", "fpc", "pu", "mu_x_top", ) if k in col_data
}

mi_idx = pd.MultiIndex.from_frame(pd.DataFrame(levels_and_loc_dict))
col_sched_df = pd.DataFrame(col_sched_dict)
col_sched_df.index = mi_idx

col_sched_df = col_sched_df.transpose()

final = []
for story in pd.Series(col_data["level"]).unique():
    story_df = col_sched_df.loc[:, story].copy()
    story_df.loc[:, "story"] = story
    final.append(story_df)
schedule_df = pd.concat(final)

schedule_df = schedule_df.reindex(schedule_df.columns.sort_values(), axis=1)
story = schedule_df.story.copy()
schedule_df = schedule_df.drop(columns=["story"])
schedule_df.insert(loc=0, column="story", value=story)


In [7]:
col_sched_df#['LEVEL 2']['(45.45ft-76.54ft)']

level        BULKHEAD ROOF                            BULKHEAD  \
grid_loc (45.45ft-76.54ft) (45.45ft-55.06ft) (24.41ft-90.91ft)   
size               16 x 24           16 x 24           16 x 24   
rebar                12-#6             12-#6             12-#6   
fpc                   6.00              6.00              6.00   
pu                   67.41             96.05             95.99   
mu_x_top            -76.93             18.80           -130.27   

level                                                              \
grid_loc (110.64ft-90.91ft) (110.83ft-76.45ft) (111.09ft-46.16ft)   
size                16 x 24            16 x 24            16 x 24   
rebar                 12-#6              12-#6              12-#6   
fpc                    6.00               6.00               6.00   
pu                    50.63              33.20              66.28   
mu_x_top             -41.44             -16.82              24.34   

level                                                            \
grid_loc (110.90ft-32.83ft) (90.83ft-90.91ft) (90.64ft-76.40ft)   
size                16 x 24           16 x 24           16 x 24   
rebar                 12-#6             12-#6             12-#6   
fpc                    6.00              6.00              6.00   
pu                    44.58             75.21             59.37   
mu_x_top              23.71            -78.28            -64.44   

level                       ...         1ST FLOOR                    \
grid_loc (91.44ft-46.16ft)  ... (30.85ft-18.85ft) (30.85ft--1.14ft)   
size               16 x 24  ...           20 x 24           16 x 24   
rebar                12-#6  ...            12-#10             12-#6   
fpc                   6.00  ...             10.00             10.00   
pu                   77.94  ...           2313.86           1376.72   
mu_x_top            -24.54  ...           -254.52           -151.44   

level                                                            \
grid_loc (24.41ft-100.17ft) (24.41ft-90.91ft) (15.81ft-18.85ft)   
size                24 x 24           24 x 24           22 x 24   
rebar                18-#11            18-#11            18-#10   
fpc                   10.00             10.00             10.00   
pu                  3061.99           2367.35           2815.67   
mu_x_top             336.82           -260.41            309.72   

level                                                          \
grid_loc (15.34ft--1.03ft) (1.21ft-100.17ft) (1.02ft-90.91ft)   
size               16 x 24           20 x 24          20 x 24   
rebar                12-#8             12-#6            12-#7   
fpc                  10.00             10.00            10.00   
pu                 1774.82           2021.68          1384.77   
mu_x_top            195.23            222.38          -152.32   

level                                         
grid_loc (-0.50ft-18.84ft) (-0.58ft--0.58ft)  
size               16 x 24           16 x 24  
rebar                12-#9             12-#8  
fpc                  10.00             10.00  
pu                 1935.74           1774.45  
mu_x_top           -212.93           -195.19  

[5 rows x 1211 columns]

In [8]:
test = schedule_df.reset_index().set_index(['story', 'index'])#.to_excel('test.xlsx')

In [9]:
test.index.names=['story', 'designs']

In [10]:
test

grid_loc               (-0.50ft-18.84ft) (-0.58ft--0.58ft) (1.02ft-90.91ft)  \
story         designs                                                         
BULKHEAD ROOF size                   NaN               NaN              NaN   
              rebar                  NaN               NaN              NaN   
              fpc                    NaN               NaN              NaN   
              pu                     NaN               NaN              NaN   
              mu_x_top               NaN               NaN              NaN   
...                                  ...               ...              ...   
1ST FLOOR     size               16 x 24           16 x 24          20 x 24   
              rebar                12-#9             12-#8            12-#7   
              fpc                  10.00             10.00            10.00   
              pu                 1935.74           1774.45          1384.77   
              mu_x_top           -212.93           -195.19          -152.32   

grid_loc               (1.21ft-100.17ft) (1.31ft-104.84ft)  \
story         designs                                        
BULKHEAD ROOF size                   NaN               NaN   
              rebar                  NaN               NaN   
              fpc                    NaN               NaN   
              pu                     NaN               NaN   
              mu_x_top               NaN               NaN   
...                                  ...               ...   
1ST FLOOR     size               20 x 24               NaN   
              rebar                12-#6               NaN   
              fpc                  10.00               NaN   
              pu                 2021.68               NaN   
              mu_x_top            222.38               NaN   

grid_loc               (110.46ft-105.17ft) (110.52ft--1.72ft)  \
story         designs                                           
BULKHEAD ROOF size                     NaN                NaN   
              rebar                    NaN                NaN   
              fpc                      NaN                NaN   
              pu                       NaN                NaN   
              mu_x_top                 NaN                NaN   
...                                    ...                ...   
1ST FLOOR     size                     NaN            16 x 24   
              rebar                    NaN              12-#6   
              fpc                      NaN              10.00   
              pu                       NaN             139.75   
              mu_x_top                 NaN             -35.52   

grid_loc               (110.53ft-100.20ft) (110.53ft-18.85ft)  \
story         designs                                           
BULKHEAD ROOF size                     NaN                NaN   
              rebar                    NaN                NaN   
              fpc                      NaN                NaN   
              pu                       NaN                NaN   
              mu_x_top                 NaN                NaN   
...                                    ...                ...   
1ST FLOOR     size                 26 x 26            24" DIA   
              rebar                 16-#11              12-#6   
              fpc                    10.00              10.00   
              pu                   3332.91             214.41   
              mu_x_top             -383.28             -42.62   

grid_loc               (110.64ft-90.91ft)  ... (74.61ft--1.72ft)  \
story         designs                      ...                     
BULKHEAD ROOF size                    NaN  ...               NaN   
              rebar                   NaN  ...               NaN   
              fpc                     NaN  ...               NaN   
              pu                      NaN  ...               NaN   
              mu_x_top                NaN  ...               NaN 

In [36]:
import webbrowser

In [37]:
webbrowser.open('test.xlsx')

True

In [11]:
from eng_module import ram_column_schedule as rcs

In [12]:
test = rcs.create_RAM_concrete_column_schedule(col_data)

In [20]:
IDX = pd.IndexSlice
test.loc[IDX["1ST FLOOR", "(-0.50ft-18.84ft)"]]

designs
size        16 x 24
rebar         12-#9
fpc           10.00
pu          1935.74
mu_x_top    -212.93
Name: (-0.50ft-18.84ft), dtype: object

In [5]:
schedule = create_RAM_concrete_column_schedule(col_data, False)

/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_81438/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_81438/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_81438/501263265.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [6]:
schedule.head()

grid_loc,story,(-0.50ft-18.84ft),(-0.58ft--0.58ft),(1.02ft-90.91ft),(1.21ft-100.17ft),(1.31ft-104.84ft),(110.46ft-105.17ft),(110.52ft--1.72ft),(110.53ft-100.20ft),(110.53ft-18.85ft),...,(74.61ft--1.72ft),(90.61ft-73.87ft),(90.64ft-76.40ft),(90.83ft-90.91ft),(90.94ft-100.20ft),(91.01ft-105.17ft),(91.44ft--1.72ft),(91.44ft-18.85ft),(91.44ft-32.83ft),(91.44ft-46.16ft)
size,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rebar,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fpc,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pu,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size,BULKHEAD,16 x 24,16 x 24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16 x 24,16 x 24,NaN,NaN,NaN,NaN,16 x 24,16 x 24


# Steel

In [2]:
raw_data = utils.read_csv_file('17717_240703_grav_col_designs.csv')
raw_data

[['Gravity Column Design'],
 ['RAM Steel'],
 ['23.00.00.92'],
 ['DataBase: 17717_240315_mtk_ahava'],
 ['Building Code: IBC'],
 ['Steel Code: AISC360-16 LRFD'],
 [' 07/03/24  10:58:25'],
 [' Story level Roof', '  Column Line 1-D1', '   Column # 4'],
 ['Fy (ksi)', '=', '  50.00', '', 'Column Size', '=', 'W10X33'],
 ['Orientation (deg.)', '=', '90.0', '', '', '', '', ''],
 ['INPUT DESIGN PARAMETERS:'],
 ['', ' X-Axis', ' Y-Axis'],
 ['Lu (ft) ..', ' 12.00', '     12.00'],
 ['K ..', '     1', '         1'],
 ['Braced Against Joint Translation .', '   Yes', '       Yes'],
 ['Column Eccentricity (in)', 'Top   ..', '  7.36', '      6.48'],
 ['', 'Bottom .', '  7.36', '      6.48'],
 ['CONTROLLING AXIAL COLUMN LOADS - Skip-Load Case 1:'],
 ['', ' Dead', ' Live', ' Roof'],
 ['Axial (kip) ..', '  15.42', '      6.39', '      0.00'],
 ['       DEMAND CAPACITY RATIO:   (1.2DL + 1.6LL)'],
 ['Pu (kip)',
  '=',
  '   28.72',
  '',
  '0.90Pnx (kip)',
  '=',
  '  400.91',
  '',
  'Pu/0.90Pnx',
  '=',
  

In [3]:
def extract_RAM_steel_grav_col_data(filepath: str) -> dict[str, list[str]]:
    """
    Returns a dictionary of relevant column design data from the RAM Concrete
    Column "Column Design" csv.
    """
    raw_data = utils.read_csv_file(filepath)

    level = []
    grid_loc = []
    col_num = []
    size = []
    lux = []
    luy = []
    kx = []
    ky = []
    pu = []

    for idx, row in enumerate(raw_data):
        # multiIndex keys
        if " Story level" in row[0]:
            level.append(row[0].split(" Story level ")[-1])
            grid_loc.append(row[1].split("  Column Line ")[-1])
            col_num.append(row[-1].strip())

        # corresponding values for above keys
        if "Fy" in row[0]:
            if "Hanging" in row[4]:
                size.append("(H) " + row[-1])
            else:
                size.append(row[-1])
        if "Lu" in row[0]:
            lux.append(row[1].strip())
            luy.append(row[-1].strip())
        if "K .." in row:
            kx.append(row[1].strip())
            ky.append(row[-1].strip())
        if "Unbraced Length (ft)." in row:
            lux.append(row[1])
            luy.append(row[2])
        if "K." in row:
            kx.append(row[1].strip())
            ky.append(row[2].strip())
        if "Pu (kip)" in row:
            pu.append(row[2].strip())
        if "Moment" in row and "Top" in row:
            mu_x_top.append(row[-1].strip())
            mu_y_top.append(raw_data[idx+1][-1].strip())
        if "Moment" in row and "Bottom" in row:
            mu_x_bot.append(row[-1].strip())
            mu_y_bot.append(raw_data[idx+1][-1].strip())

    column_data = {
        "level": level,
        "grid_loc": grid_loc,
        "size": size,
        "lux": lux,
        "luy": luy,
        "kx": kx,
        "ky": ky,
        "pu": pu,
    }

    print(len(level), len(grid_loc), len(col_num), len(size), len(lux),
          len(luy), len(kx), len(ky), len(pu))

    return column_data

In [4]:
extract_RAM_steel_grav_col_data('17717_240703_grav_col_designs.csv')

84 84 84 84 84 84 84 84 84


{'level': ['Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second',
  'Roof',
  'Third',
  'Second'],
 'grid_loc': ['1-D1',
  '1-D1',
  '1-D1',
  '1-A',
  '1-A',
  '1-A

In [14]:
def create_RAM_steel_grav_column_schedule(column_data: dict[str, list[str]],
                    xlsx: bool = False,
                    output_filename: str = "RAM_Gravity_Steel_Column_Schedule.xlsx") -> pd.DataFrame:
    """
    Returns a column schedule from the dictionary created from the
    extract_RAM_steel_grav_column_data() function.

    If xlsx = True, it will also produce an Excel file.
    """
    levels_and_loc_dict = {k: column_data[k] for k in ('level', 'grid_loc') if k in column_data}
    col_sched_dict = {k: column_data[k] for k in ('size', 'pu') if k in column_data}

    mi_idx = pd.MultiIndex.from_frame(pd.DataFrame(levels_and_loc_dict))
    col_sched_df = pd.DataFrame(col_sched_dict)
    col_sched_df.index = mi_idx

    col_sched_df = col_sched_df.transpose()

    final = []
    for story in pd.Series(column_data['level']).unique():
        story_df = col_sched_df.loc[:, story]
        story_df['story'] = story
        final.append(story_df)
    schedule_df = pd.concat(final)


    schedule_df = schedule_df.reindex(schedule_df.columns.sort_values(), axis=1)
    story = schedule_df.story
    schedule_df = schedule_df.drop(columns=['story'])
    schedule_df.insert(loc=0, column='story', value=story)

    if xlsx:
        schedule_df.to_excel(output_filename)

    return schedule_df

In [15]:
steel_grav_col_data = extract_RAM_steel_grav_col_data('17717_240703_grav_col_designs.csv')

84 84 84 84 84 84 84 84 84


In [18]:
steel_grav_schedule = create_RAM_steel_grav_column_schedule(steel_grav_col_data, True)

/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_91022/3147637095.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_91022/3147637095.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_91022/3147637095.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [17]:
steel_grav_schedule

grid_loc,story,1-A,1-D1,104.52ft-63.92ft,126.10ft-63.92ft,2-A,2-B,2-D1,2A-D1,3-B,...,5-C1,5-D1,6-B,6-C1,6-D1,7-A,7-D1,8-A,8-D1,92.60ft-63.92ft
size,Roof,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,...,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33
pu,Roof,13.30,28.72,25.94,12.23,23.58,54.30,11.80,22.70,69.55,...,68.10,39.55,65.79,80.98,46.77,19.41,37.96,8.16,12.92,25.01
size,Third,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,...,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33
pu,Third,49.27,92.32,79.77,46.47,72.06,135.12,42.55,67.65,175.22,...,171.47,118.28,165.23,205.68,141.76,55.67,112.94,32.99,49.76,79.36
size,Second,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X39,...,W10X33,W10X33,W10X39,W10X39,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33
pu,Second,85.24,157.65,133.65,80.76,120.54,215.94,73.30,99.12,281.26,...,274.88,197.01,264.78,330.45,236.75,91.98,187.93,57.81,86.59,133.72


In [47]:
def extract_RAM_grav_baseplate_data(
    filepath: str, debug: bool = False
) -> dict[str, list[str]]:
    """
    Returns a dictionary of relevant column design data from the RAM Steel
    Column Baseplate csv.

    Assumes baseplates are square.
    """
    raw_data = utils.read_csv_file(filepath)

    grid_loc = []
    size_n = []
    dead_load = [] 
    live_load = []
    pl_thickness = []
    col_size = []

    for row in raw_data:
        if 'Base Plate level ' in row[0]:
            grid_loc.append(row[-1].split(' ')[-1])
        
        # square base plates
        if 'N (parallel to web) (in) .' in row[0]:
            size_n.append(row[-1] + 'x'  + row[-1])

        if 'Plate Thickness (in) .' == row[0]:
            pl_thickness.append(row[-1].strip())

        if 'Column Size .' in row[0]:
            col_size.append(row[-1])

        if 'Dead Load (kips) . ' in row[0]:
            dead_load.append(float(row[-1].strip()))
        
        if 'Live Load (kips) .' in row[0]:
            live_load.append(float(row[-1].strip()))

    pa = np.array(dead_load) + np.array(live_load)

    baseplate_data = {
        "grid_loc": grid_loc,
        "bp_size": size_n,
        "pl_thickness": pl_thickness,
        "col_size": col_size,
        "pa": pa
    }
    if debug:
        print(
            f'grid_loc: {len(grid_loc)}',
            f'bp_size: {len(size_n)}',
            f'col_size: {len(col_size)}',
            f'pa: {len(pa)}',
        )

    return baseplate_data

def create_RAM_grav_baseplate_schedule(
        baseplate_data: dict[str, list[str]],
        xlsx: bool = False, 
        output_filename: str = "RAM_Gravity_Baseplate_Schedule.xlsx"
        ) -> pd.DataFrame:
    """
    Returns the gravity baseplate schedule from the dictionary created from the
    extract_RAM_grav_baseplate_data() function.
    
    If xlsx = True, it will also produce an Excel file.
    """
    loc_dict = {k: baseplate_data[k] for k in ('grid_loc') if k in baseplate_data}
    bp_sched_dict = {k: baseplate_data[k] for k in ('grid_loc', 'col_size', 'bp_size', 'pl_thickness', 'pa') if k in baseplate_data}

    # mi_idx = pd.MultiIndex.from_frame(pd.DataFrame(loc_dict))
    bp_df = pd.DataFrame.from_dict(bp_sched_dict)
    # bp_df.index = mi_idx

    bp_df = bp_df.transpose()
    return bp_df

In [48]:
bp_dict = extract_RAM_grav_baseplate_data('17717_240703_grav_base_plate_designs.csv')

In [49]:
create_RAM_grav_baseplate_schedule(bp_dict, True)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
grid_loc,1-D1,1-A,2-D1,2-B,2-A,2A-D1,30.42ft-63.92ft,3-C1,3-B,41.58ft-63.92ft,...,92.60ft-63.92ft,104.52ft-63.92ft,6-D1,6-C1,6-B,126.10ft-63.92ft,7-D1,7-A,8-D1,8-A
col_size,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X45,W10X39,W10X33,...,W10X33,W10X33,W10X33,W10X39,W10X39,W10X33,W10X33,W10X33,W10X33,W10X33
bp_size,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,19.00x 19.00,18.00x 18.00,18.00x 18.00,...,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00
pl_thickness,1.000,0.750,0.750,1.250,0.750,0.750,0.750,1.500,1.250,1.000,...,1.000,1.000,1.250,1.250,1.250,0.750,1.000,0.750,0.750,0.750
pa,121.99,66.82,57.36,161.08,92.81,76.67,49.18,271.88,210.1,114.73,...,103.33,102.97,181.56,246.94,197.76,63.43,144.01,70.44,68.09,45.65


In [52]:
from eng_module import ram_column_schedule as rcs

In [55]:
bp_dict = rcs.extract_RAM_grav_baseplate_data('17717_240703_grav_base_plate_designs.csv')

In [56]:
rcs.create_RAM_grav_baseplate_schedule(bp_dict)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
grid_loc,1-D1,1-A,2-D1,2-B,2-A,2A-D1,30.42ft-63.92ft,3-C1,3-B,41.58ft-63.92ft,...,92.60ft-63.92ft,104.52ft-63.92ft,6-D1,6-C1,6-B,126.10ft-63.92ft,7-D1,7-A,8-D1,8-A
col_size,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X33,W10X45,W10X39,W10X33,...,W10X33,W10X33,W10X33,W10X39,W10X39,W10X33,W10X33,W10X33,W10X33,W10X33
bp_size,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,19.00x 19.00,18.00x 18.00,18.00x 18.00,...,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00,18.00x 18.00
pl_thickness,1.000,0.750,0.750,1.250,0.750,0.750,0.750,1.500,1.250,1.000,...,1.000,1.000,1.250,1.250,1.250,0.750,1.000,0.750,0.750,0.750
pa,121.99,66.82,57.36,161.08,92.81,76.67,49.18,271.88,210.1,114.73,...,103.33,102.97,181.56,246.94,197.76,63.43,144.01,70.44,68.09,45.65


In [59]:
conc_dict = rcs.extract_RAM_conc_column_data(
    "/Users/danki/pycode/eng_module/test_data/RAM_Lateral_Concrete_Designs.csv"
)

In [60]:
conc_df = rcs.create_RAM_concrete_column_schedule(conc_dict)

/Users/danki/pycode/eng_module/ram_column_schedule.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df["story"] = story
/Users/danki/pycode/eng_module/ram_column_schedule.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df["story"] = story
/Users/danki/pycode/eng_module/ram_column_schedule.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [73]:
conc_df.tail()

grid_loc,story,(-0.50ft-18.84ft),(-0.58ft--0.58ft),(1.02ft-90.91ft),(1.21ft-100.17ft),(1.31ft-104.84ft),(110.46ft-105.17ft),(110.52ft--1.72ft),(110.53ft-100.20ft),(110.53ft-18.85ft),...,(74.61ft--1.72ft),(90.61ft-73.87ft),(90.64ft-76.40ft),(90.83ft-90.91ft),(90.94ft-100.20ft),(91.01ft-105.17ft),(91.44ft--1.72ft),(91.44ft-18.85ft),(91.44ft-32.83ft),(91.44ft-46.16ft)
pu,1ST MEZZANINE,1872.68,1716.74,1325.29,1965.84,NaN,NaN,75.75,3228.84,118.82,...,99.43,2257.46,NaN,NaN,3974.11,NaN,83.21,129.14,1696.83,2693.54
size,1ST FLOOR,16 x 24,16 x 24,20 x 24,20 x 24,NaN,NaN,16 x 24,26 x 26,"24"" DIA",...,16 x 24,24 x 24,NaN,NaN,28 x 28,NaN,16 x 24,"24"" DIA","28"" DIA",20 x 24
rebar,1ST FLOOR,12-#9,12-#8,12-#7,12-#6,NaN,NaN,12-#6,16-#11,12-#6,...,12-#6,12-#7,NaN,NaN,24-#11,NaN,12-#6,12-#6,12-#7,18-#11
fpc,1ST FLOOR,10.00,10.00,10.00,10.00,NaN,NaN,10.00,10.00,10.00,...,10.00,10.00,NaN,NaN,10.00,NaN,10.00,10.00,10.00,10.00
pu,1ST FLOOR,1935.74,1774.45,1384.77,2021.68,NaN,NaN,139.75,3332.91,214.41,...,185.93,2337.75,NaN,NaN,4110.90,NaN,153.96,236.56,1777.34,2789.92


In [83]:
conc_df.loc['size','(-0.50ft-18.84ft)']

size        NaN
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
size    16 x 24
Name: (-0.50ft-18.84ft), dtype: object